# Backward stability of solving $Ax=b$ by Householder QR factorization

In this notebook we demonstrate the backward stability of solving $Ax=b$ by the following algorithm:

1. Factor $A=QR$ using Householder triangularization.
2. Compute $y=Q^* b$.
3. Solve $Ry = x$ by backward substitution.

In [ ]:
import numpy as np

First, let's implement Householder (*note: we don't need to actually compute $Q$ in the algorithm above, so we could just apply $Q^*$ directly using algorithm 10.2 from Trefethen & Bau*).

In [ ]:
def householder(A):
    """QR factorization via Householder triangularization."""
    m, n = A.shape
    V = np.zeros(A.shape)
    R = A.copy()
    for k in range(n-1):
        x = R[k:,k].copy()
        x[0] = x[0] + np.sign(x[0])*np.linalg.norm(x,2)
        x = x/np.linalg.norm(x,2)
        V[k:,k] = x.copy()
        for j in range(k,n):
            R[k:,j] = R[k:,j] - 2*V[k:,k]*np.dot(V[k:,k].T,R[k:,j])
    return V,R[:n,:]

def apply_Q(V,x):
    """Algorithm 10.3 of Trefethen & Bau."""
    m, n = V.shape
    for k in range(n-1,-1,-1):
        x[k:] = x[k:] - 2*np.dot(V[k:,k],x[k:])*V[k:,k]
    return x

def compute_Q(V):
    """Find Q given the Householder reflector vectors."""
    m, n = V.shape
    Q = np.zeros((m,n))
    for k in range(n):
        x = np.zeros(m)
        x[k] = 1.
        Q[:,k] = apply_Q(V,x)
    return Q

In order to show stability, let's pick a really nasty matrix:

In [ ]:
def Hilbert(n):
    """Return the n x n Hilbert matrix."""
    A = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            A[i,j] = 1./(i+j+1)
    return A

In [ ]:
N=10
x = np.ones(N)
A = Hilbert(N)
b = np.dot(A,x)

In [ ]:
kappa = np.linalg.cond(A)
np.log10(kappa)

The condition number is about $10^{13}$, which means we could lose 13 digits of accuracy just due to rounding of the inputs.  Hence a backward stable algorithm can be expected to give us $16-13 = 3$ accurate digits in double precision.

In [ ]:
V, R = householder(A)
Q = compute_Q(V)
y = np.dot(Q.T,b)
x_tilde = np.linalg.solve(R,y)

In [ ]:
print(np.linalg.norm(x_tilde-x))

Indeed, we are left with about 3 significant accurate digits; in other words, the forward error is about $10^{-3}$.

In [ ]:
print(x_tilde)

How large is the backward error?

In [ ]:
norm = np.linalg.norm
r = np.dot(A,x_tilde)-b
print(norm(r)/norm(A)*norm(x_tilde))

Still on the order of unit roundoff, as we expect.